Name: 이운규

ID: 2016320145

Link to your midterm project: https://colab.research.google.com/drive/1_ShNVJovC-oJ4nq52ObupMA0eo8geetV?usp=sharing

Link to your evaluation script: https://colab.research.google.com/drive/1LlKl_8uCnjwjc5LMd0zm2O9oSr-Ub4lV?usp=sharing

These links should be shared to the web without any permission

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn import metrics
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV,RandomizedSearchCV
from google.colab import drive
from pandas import DataFrame
drive.mount('/content/drive/')
a =0
filename = '/content/drive/My Drive/뇌및머신러닝/train_data.csv'
data = pd.read_csv(filename)

data.head()

evaluation_data_path = '/content/drive/My Drive/뇌및머신러닝/sample_evaluation_data.csv'
cls_weight_path = '/content/drive/My Drive/뇌및머신러닝/cls_model.pkl'
reg_weight_path1 = '/content/drive/My Drive/뇌및머신러닝/reg_model_1.pkl'
reg_weight_path2 = '/content/drive/My Drive/뇌및머신러닝/reg_model_2.pkl'
reg_weight_path3 = '/content/drive/My Drive/뇌및머신러닝/reg_model_3.pkl'

# data = pd.read_csv(evaluation_data_path)
X=data.drop(columns=['DX_bl','ADAS11','ADAS13','MMSE'],axis=1) #Predictors
# y=data['DX_bl'] #Response
col_list=X.columns.values.tolist()
print(col_list)
# X = X.interpolate(method='linear')
# y1=data['ADAS11'] #Response
# y2=data['ADAS13']
# y3=data['MMSE']
#X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.1,random_state = 4)


svc = pickle.load(open(cls_weight_path, 'rb'))

rfr1 = pickle.load(open(reg_weight_path1, 'rb'))
rfr2 = pickle.load(open(reg_weight_path2, 'rb'))
rfr3 = pickle.load(open(reg_weight_path3, 'rb'))
# rfr_pred= rfr.predict(X)
# print(rfr_pred)
# print('RandomForestRegressor')
# print('MAE:', metrics.mean_absolute_error(y_test, rfr_pred))
# print('MSE:', metrics.mean_squared_error(y_test, rfr_pred))
# print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, rfr_pred)))

# score=cross_val_score(rfr,X_train,y_train,cv=10)
# print(score)
# avscore = np.mean(score)
# print('average train score: ',avscore)

def load_best_classifier():
    model = svc
    
    # Write your load function for classifier model here
    # model = 

    return model

def load_best_regressor():
    model = (rfr1,rfr2,rfr3)

    # Write your load function for regression model here
    # model = 

    return model 

def read_csv_and_classify(eval_data, cls_model):
    eval_data = eval_data[:,:140]
    eval_data = pd.DataFrame(data=eval_data,columns=col_list)
    eval_data=eval_data.interpolate()
    eval_data=eval_data[['ST116CV', 'ST23CV', 'ST24CV', 'ST44CV', 'ST83CV', 'ST91CV', 'ST24TA',
       'ST31TA', 'ST32TA', 'ST35TA', 'ST40TA', 'ST64TA', 'ST83TA', 'ST91TA',
       'ST99TA']]
    #print(eval_data)
    scores = np.zeros(shape=len(eval_data), dtype=np.uint8)
    scores = cls_model.predict(eval_data)
    print(scores.dtype)
    scores = scores.astype(np.uint8)
    # Write your evaluation script here
    # scores = 

    return scores # Should be a numpy array with shape=(len(eval_data), 1) and data type=np.uint8

def read_csv_and_regress(eval_data, reg_model):
    eval_data = eval_data[:,:140]
    eval_data = pd.DataFrame(data=eval_data,columns=col_list)
    eval_data=eval_data.interpolate()
    # eval_data=eval_data[['ST116CV', 'ST23CV', 'ST24CV', 'ST44CV', 'ST83CV', 'ST91CV', 'ST24TA',
    #    'ST31TA', 'ST32TA', 'ST35TA', 'ST40TA', 'ST64TA', 'ST83TA', 'ST91TA',
    #    'ST99TA']]
    scores = np.zeros(shape=(len(eval_data), 3), dtype=np.float32)
    
    scores1 = reg_model[0].predict(eval_data)
    print(scores1)
    
    scores2 = reg_model[1].predict(eval_data)
   
    scores3 = reg_model[2].predict(eval_data)
    scores = np.array([scores1,scores2,scores3])
   
    scores = scores.reshape(3, len(eval_data))
    scores = scores.T
    scores = scores.astype(np.float32)
    
    # Write your evaluation script here
    # scores = 

    return scores # Should be a numpy array with shape=(len(eval_data), 3) and data type=np.float32

import os
import numpy as np
from google.colab import files
 
# Load Data
with open(evaluation_data_path, "r") as f:
    eval_lines = f.read().strip().split("\n")
    eval_header = eval_lines[0]
    eval_data = eval_lines[1:]
eval_data = np.array([[ff  if ff!="" else "nan" for ff in fff.strip().split(",")] for fff in eval_data], dtype=np.float32)
 
# Load Weights
best_classification_model = load_best_classifier()
best_regression_model = load_best_regressor()
 
# Calculate scores
cls_scores = read_csv_and_classify(eval_data[:, 4:], best_classification_model)
reg_scores = read_csv_and_regress(eval_data[:, 4:], best_regression_model)
 
# Check scores
assert cls_scores.shape==(len(eval_data),), "Classification score shape mismatch"
assert reg_scores.shape==(len(eval_data),3), "Regression score shape mismatch"
assert cls_scores.dtype==np.uint8, "Classification score data type mismatch"
assert reg_scores.dtype==np.float32, "Regression score data type mismatch"
 
# Save and download scores
eval_data[:, 0] = cls_scores
eval_data[:, 1:4] = reg_scores
 
with open(evaluation_data_path, "w") as f:
    f.write(eval_header+"\n"+"\n".join([",".join([str(ff) for ff in fff]) for fff in eval_data]))
 
files.download(evaluation_data_path)